In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session





!pip install torch torchvision
!pip install 'git+https://github.com/facebookresearch/detectron2.git'


  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-9p20n8k8
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-9p20n8k8
  Resolved https://github.com/facebookresearch/detectron2.git to commit b7c7f4ba82192ff06f2bbb162b9f67b00ea55867
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 877.2 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 1.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 7.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import json
import cv2
import numpy as np
import torch

from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import functional as F
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.structures import BoxMode


# Emptying cuda memory 
torch.cuda.empty_cache()
import gc
torch.cuda.empty_cache()
gc.collect()

# COCO Dataset class
# COCO Dataset class
class COCODataset(Dataset):
    def __init__(self, annotations_file, image_dir, transforms=None):
        self.transforms = transforms
        self.image_dir = image_dir
        with open(annotations_file, 'r') as f:
            self.data = json.load(f)
    
    def __len__(self):
        return len(self.data['images'])
    
    def __getitem__(self, idx):
        img_info = self.data['images'][idx]
        image_path = os.path.join(self.image_dir, img_info['file_name'])
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Find annotations for this image
        annotations = [anno for anno in self.data['annotations'] if anno['image_id'] == img_info['id']]
        
        
        target = {}
        target['boxes'] = []
        target['labels'] = []
        for anno in annotations:
            bbox = anno['bbox']
            target['boxes'].append([bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]])  # Convert to [x1, y1, x2, y2]
            target['labels'].append(anno['category_id'])
        
        target['boxes'] = torch.tensor(target['boxes'], dtype=torch.float32)
        target['labels'] = torch.tensor(target['labels'], dtype=torch.int64)


        # for all image files whose annotations are not available, assign a dummy label
        if len(annotations) == 0:
            target['boxes'] = torch.tensor([[0, 0, 1, 1]], dtype=torch.float32)
            target['labels'] = torch.tensor([1], dtype=torch.int64)
        
        if self.transforms:
            image, target = self.transforms(image, target)
        
        return image, target


# YOLO Dataset class
class YOLODataset(Dataset):
    def __init__(self, image_dir, label_dir, transforms=None):
        self.transforms = transforms
        self.image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir)]
        self.label_dir = label_dir
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        label_file = os.path.join(self.label_dir, os.path.basename(img_path).replace('.png', '.txt'))
        targets = []
        # check if label file exists
        if not os.path.exists(label_file):
            # add the label as '1' to indicate no object 
            targets.append({'boxes': [[0, 0, 1, 1]], 'labels': [1]})
            return image, targets
        
        with open(label_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                class_label = int(parts[0])
                x_center, y_center, width, height = map(float, parts[1:])
                # Convert normalized coordinates to absolute coordinates
                img_width, img_height = image.shape[1], image.shape[0]
                x1 = int((x_center - width / 2) * img_width)
                y1 = int((y_center - height / 2) * img_height)
                x2 = int((x_center + width / 2) * img_width)
                y2 = int((y_center + height / 2) * img_height)
                targets.append({'boxes': [[x1, y1, x2, y2]], 'labels': [class_label]})
        
        if self.transforms:
            image, targets = self.transforms(image, targets)
        
        return image, targets

# # Transform for COCO Dataset
# def transform_coco(image, target):
#     image = F.to_tensor(image)
#     target['boxes'] = torch.tensor(target['bbox'], dtype=torch.float32)
#     return image, target

# Transform for COCO Dataset
def transform_coco(image, target):
    image = F.to_tensor(image)
    target['boxes'] = target['boxes']  # Already in correct format
    return image, target

# Transform for YOLO Dataset
def transform_yolo(image, targets):
    image = F.to_tensor(image)
    return image, targets

# Function to collate batch data
def collate_fn(batch):
    return tuple(zip(*batch))

# Define training loop for Faster R-CNN
def train_faster_rcnn(train_data_loader, val_data_loader, num_epochs=5):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model = fasterrcnn_resnet50_fpn(pretrained=True)
    model.to(device)
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
    # Supreeti - Add any Other/modify training configurations if u want
    for epoch in range(num_epochs):
    
        model.train()
        for images, targets in train_data_loader:
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()


        # print training metrics
        print(f"Epoch {epoch} Training Loss: {losses.item()}")

        # Validation loop and logging
        model.eval()
        for images, targets in val_data_loader:
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
        # print validation metrics
        print(f"Epoch {epoch} Validation Loss: {losses.item()}")
        

    
# Define training loop for Deformable DETR
def train_deformable_detr(train_data_loader, val_data_loader, num_epochs=5):
    # Configuration for training Deformable DETR
    cfg = get_cfg()
    cfg.merge_from_file("configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
    cfg.DATASETS.TRAIN = ("train",)
    cfg.DATASETS.TEST = ("val",)
    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.MODEL.WEIGHTS = "detectron2://ImageNetPretrained/MSRA/R-50.pkl"
    cfg.SOLVER.IMS_PER_BATCH = 2
    cfg.SOLVER.BASE_LR = 0.0025
    cfg.SOLVER.MAX_ITER = 5000
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 80
    
    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()

# Define dataset and data loaders
annotations_file_coco = '/kaggle/input/mammo-1k/coco_1k/annotations/instances_train2017.json'
image_dir_coco = '/kaggle/input/mammo-1k/coco_1k/train2017'
image_dir_yolo = '/kaggle/input/mammo-1k/yolo_1k/train/images'
label_dir_yolo = '/kaggle/input/mammo-1k/yolo_1k/train/labels'

coco_dataset = COCODataset(annotations_file_coco, image_dir_coco, transforms=transform_coco)
yolo_dataset = YOLODataset(image_dir_yolo, label_dir_yolo, transforms=transform_yolo)

coco_data_loader = DataLoader(coco_dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)
yolo_data_loader = DataLoader(yolo_dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)

# for data in coco_data_loader:
#     print(data)
#     break

# for data in yolo_data_loader:
#     print(data)
#     break
    

# print the length of the data loaders
print("Number of images in COCO data folder:", len(os.listdir(image_dir_coco)))
print("Number of images in YOLO data folder:", len(os.listdir(image_dir_yolo)))
print("Number of batches in COCO dataloader:", len(coco_data_loader))
print("Number of batches in YOLO dataloader:", len(yolo_data_loader))


# Train Faster R-CNN
train_faster_rcnn(coco_data_loader, coco_data_loader)

# Train Deformable DETR
train_deformable_detr(yolo_data_loader, yolo_data_loader)



    
    


Number of images in COCO data folder: 2240
Number of images in YOLO data folder: 2240
Number of batches in COCO dataloader: 2240
Number of batches in YOLO dataloader: 2240
